In [64]:
from neo4j import GraphDatabase
import os

def create_issue_creator_relationship(driver, user_github_id, issue_owner, issue_repo_name, issue_number):
    with driver.session(database="neo4j") as session:
        session.write_transaction(
            _create_issue_creator_relationship, user_github_id, issue_owner, issue_repo_name, issue_number
        )

def _create_issue_creator_relationship(tx, user_github_id, issue_owner, issue_repo_name, issue_number):
    query = (
        "MATCH (u:User {GithubId: $user_github_id}), "
        "(i:Issue {Owner: $issue_owner, RepoName: $issue_repo_name, IssueNumber: $issue_number}) "
        "MERGE (u)-[rel:CREATED]->(i) "
        "ON CREATE SET rel.uid = u.GithubId + '-' + i.Owner + '-' + i.RepoName + '-' + toString(i.IssueNumber)"
    )
    tx.run(
        query,
        user_github_id=user_github_id,
        issue_owner=issue_owner,
        issue_repo_name=issue_repo_name,
        issue_number=issue_number,
    )

In [65]:
uri = "neo4j+s://1b845d2a.databases.neo4j.io"
user = os.environ.get('NEO4J_USERNAME')
password = os.environ.get('NEO4J_PASSWORD')
driver = GraphDatabase.driver(uri, auth=(user, password))

In [66]:
create_issue_creator_relationship(driver, "hindupurv", "BigDataIA-Spring2023-Team-04", "BigDataIA-Assignment-03", 6)

C:\Users\Karan\AppData\Local\Temp\ipykernel_7912\248458065.py:6: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(


In [67]:
def create_issue_assigner_relationship(driver, assigner_github_id, assignee_github_id, issue_owner, issue_repo_name, issue_number):
    with driver.session(database="neo4j") as session:
        session.write_transaction(
            _create_issue_assigner_relationship, assigner_github_id, assignee_github_id, issue_owner, issue_repo_name, issue_number
        )

def _create_issue_assigner_relationship(tx, assigner_github_id, assignee_github_id, issue_owner, issue_repo_name, issue_number):
    query = (
        "MATCH (u1:User {GithubId: $assigner_github_id}), "
        "(u2:User {GithubId: $assignee_github_id}), "
        "(i:Issue {Owner: $issue_owner, RepoName: $issue_repo_name, IssueNumber: $issue_number}) "
        "MERGE (u1)-[rel:ASSIGNED_ISSUE]->(u2)-[:ASSIGNED_TO]->(i) "
        "ON CREATE SET rel.uid = u1.GithubId + '-' + u2.GithubId + '-' + i.Owner + '-' + i.RepoName + '-' + toString(i.IssueNumber), rel.Timestamp = timestamp()"
    )
    tx.run(
        query,
        assigner_github_id=assigner_github_id,
        assignee_github_id=assignee_github_id,
        issue_owner=issue_owner,
        issue_repo_name=issue_repo_name,
        issue_number=issue_number,
    )

In [68]:
create_issue_assigner_relationship(driver, "KaranAgrawal1509", "KaranAgrawal1509", "BigDataIA-Spring2023-Team-04", "BigDataIA-Assignment-03", 6)
create_issue_assigner_relationship(driver, "KaranAgrawal1509", "LokeshwaranVenugopalBalamurugan", "BigDataIA-Spring2023-Team-04", "BigDataIA-Assignment-03", 6)
create_issue_assigner_relationship(driver, "SivaranjaniSuresh", "SivaranjaniSuresh", "BigDataIA-Spring2023-Team-04", "BigDataIA-Assignment-03", 6)

C:\Users\Karan\AppData\Local\Temp\ipykernel_7912\1287477064.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(
C:\Users\Karan\AppData\Local\Temp\ipykernel_7912\1287477064.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(
C:\Users\Karan\AppData\Local\Temp\ipykernel_7912\1287477064.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(


In [69]:
def user_reacted_on_issue(driver, github_id, issue_owner, issue_repo_name, issue_number, reaction_name):
    with driver.session(database="neo4j") as session:
        session.write_transaction(
            _create_user_reaction_relationship, github_id, issue_owner, issue_repo_name, issue_number, reaction_name
        )

def _create_user_reaction_relationship(tx, github_id, issue_owner, issue_repo_name, issue_number, reaction_name):
    query = (
        "MATCH (u:User {GithubId: $github_id}), "
        "(i:Issue {Owner: $issue_owner, RepoName: $issue_repo_name, IssueNumber: $issue_number}), "
        "(r:Reaction {ReactionName: $reaction_name}) "
        "MERGE (u)-[rel:REACTED]->(r)-[:REACTION_ON]->(i) "
        "ON CREATE SET rel.uid = u.GithubId + '-' + i.Owner + '-' + i.RepoName + '-' + toString(i.IssueNumber) + '-' + r.ReactionName, rel.Timestamp = timestamp()"
    )
    tx.run(
        query,
        github_id=github_id,
        issue_owner=issue_owner,
        issue_repo_name=issue_repo_name,
        issue_number=issue_number,
        reaction_name=reaction_name,
    )

In [70]:
user_reacted_on_issue(driver, "KaranAgrawal1509", "BigDataIA-Spring2023-Team-04", "BigDataIA-Assignment-03", 6, "+1")
user_reacted_on_issue(driver, "SivaranjaniSuresh", "BigDataIA-Spring2023-Team-04", "BigDataIA-Assignment-03", 6, "rocket")

C:\Users\Karan\AppData\Local\Temp\ipykernel_7912\1269380029.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(
C:\Users\Karan\AppData\Local\Temp\ipykernel_7912\1269380029.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(


In [71]:
def user_added_milestone(driver, github_id, issue_owner, issue_repo_name, issue_number, milestone_type):
    with driver.session(database="neo4j") as session:
        session.write_transaction(
            _create_user_milestone_relationship, github_id, issue_owner, issue_repo_name, issue_number, milestone_type
        )

def _create_user_milestone_relationship(tx, github_id, issue_owner, issue_repo_name, issue_number, milestone_type):
    query = (
        "MATCH (u:User {GithubId: $github_id}), "
        "(i:Issue {Owner: $issue_owner, RepoName: $issue_repo_name, IssueNumber: $issue_number}), "
        "(m:Milestone {MilestoneType: $milestone_type}) "
        "MERGE (u)-[rel:ADDED_MILESTONE]->(m)-[:MILESTONE_ON]->(i) "
        "ON CREATE SET rel.uid = u.GithubId + '-' + i.Owner + '-' + i.RepoName + '-' + toString(i.IssueNumber) + '-' + m.MilestoneType, rel.Timestamp = timestamp()"
    )
    tx.run(
        query,
        github_id=github_id,
        issue_owner=issue_owner,
        issue_repo_name=issue_repo_name,
        issue_number=issue_number,
        milestone_type=milestone_type,
    )

In [72]:
user_added_milestone(driver, "KaranAgrawal1509", "BigDataIA-Spring2023-Team-04", "BigDataIA-Assignment-03", 6, "Done")

C:\Users\Karan\AppData\Local\Temp\ipykernel_7912\1586132349.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(


In [73]:
def issue_has_label(driver, issue_owner, issue_repo_name, issue_number, label_type):
    with driver.session(database="neo4j") as session:
        session.write_transaction(
            _create_issue_label_relationship, issue_owner, issue_repo_name, issue_number, label_type
        )

def _create_issue_label_relationship(tx, issue_owner, issue_repo_name, issue_number, label_type):
    query = (
        "MATCH (i:Issue {Owner: $issue_owner, RepoName: $issue_repo_name, IssueNumber: $issue_number}), "
        "(l:Label {LabelType: $label_type}) "
        "MERGE (i)-[rel:HAS_LABEL]->(l) "
        "ON CREATE SET rel.uid = i.Owner + '-' + i.RepoName + '-' + toString(i.IssueNumber) + '-' + l.LabelType"
    )
    tx.run(
        query,
        issue_owner=issue_owner,
        issue_repo_name=issue_repo_name,
        issue_number=issue_number,
        label_type=label_type,
    )

In [74]:
issue_has_label(driver, "BigDataIA-Spring2023-Team-04", "BigDataIA-Assignment-03", 6, "enhancement")

C:\Users\Karan\AppData\Local\Temp\ipykernel_7912\3555707302.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(
